In [ ]:
!pip install google-cloud-bigquery


In [ ]:
!pip install google_auth

In [ ]:
import pandas as pd
import numpy as np
import datetime
from google.cloud import bigquery
from datetime import datetime

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving data-avatar-409500-7dec0f050905.json to data-avatar-409500-7dec0f050905.json


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/data-avatar-409500-7dec0f050905.json"


In [ ]:
queries =  [
        "select sum((lo_extendedprice*lo_discount)) as revenue from data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.date where d_yearmonthnum = 199401 and lo_orderdate = d_datekey and lo_discount between 4 and 6 and lo_quantity between 26 and 35;",
        "select sum((lo_extendedprice*lo_discount)) as revenue from data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.date where d_year = 1993 and lo_orderdate = d_datekey and lo_discount between 1 and 3 and lo_quantity < 25;",
        "select sum((lo_extendedprice*lo_discount)) as revenue from data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.date where d_weeknuminyear = 6 and lo_orderdate = d_datekey and d_year = 1994 and lo_discount between 5 and 7 and lo_quantity between 26 and 35;",
        "select sum((lo_revenue)), d_year, p_brand from data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.date, data-avatar-409500.ssbsf10.part, data-avatar-409500.ssbsf10.supplier where p_category = 'MFGR#12' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey and s_region = 'AMERICA' group by d_year, p_brand order by d_year, p_brand;",
        "select sum((lo_revenue)), d_year, p_brand from data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.date, data-avatar-409500.ssbsf10.part, data-avatar-409500.ssbsf10.supplier where p_brand BETWEEN 'MFGR#2221' and  'MFGR#2228' and s_region = 'ASIA' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey group by d_year, p_brand order by d_year, p_brand;",
        "select sum((lo_revenue)), d_year, p_brand from data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.date, data-avatar-409500.ssbsf10.part, data-avatar-409500.ssbsf10.supplier where p_brand = 'MFGR#2221' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey and s_region = 'EUROPE' group by d_year, p_brand order by d_year, p_brand;",
        "select c_nation, s_nation, d_year, sum((lo_revenue)) as revenue from data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.date where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and c_region = 'ASIA' and s_region = 'ASIA' and d_year >= 1992 and d_year <= 1997 group by c_nation, s_nation, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum((lo_revenue)) as revenue from data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.date where c_nation = 'UNITED STATES' and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and s_nation = 'UNITED STATES' and d_year >= 1992 and d_year <= 1997 group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum((lo_revenue)) as revenue from data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.date where d_year >= 1992 and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') and d_year <= 1997 group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum((lo_revenue)) as revenue from data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.lineorder, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.date where d_yearmonth = 'Dec1997' and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select d_year, c_nation, sum((lo_revenue - lo_supplycost)) as profit from data-avatar-409500.ssbsf10.date, data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.part, data-avatar-409500.ssbsf10.lineorder where lo_custkey = c_custkey  and lo_suppkey = s_suppkey  and lo_partkey = p_partkey  and lo_orderdate = d_datekey  and c_region = 'AMERICA'  and s_region = 'AMERICA'  and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_year, c_nation order by d_year, c_nation;",
        "select d_year, s_nation, p_category, sum((lo_revenue - lo_supplycost)) as profit from data-avatar-409500.ssbsf10.date, data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.part, data-avatar-409500.ssbsf10.lineorder where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_partkey = p_partkey and lo_orderdate = d_datekey and c_region = 'AMERICA' and s_region = 'AMERICA' and (d_year = 1997 or d_year = 1998) and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_year, s_nation, p_category order by d_year, s_nation, p_category;",
        "select d_year, s_city, p_brand, sum((lo_revenue - lo_supplycost)) as profit from data-avatar-409500.ssbsf10.date, data-avatar-409500.ssbsf10.customer, data-avatar-409500.ssbsf10.supplier, data-avatar-409500.ssbsf10.part, data-avatar-409500.ssbsf10.lineorder where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_partkey = p_partkey and lo_orderdate = d_datekey and c_region = 'AMERICA' and s_nation = 'UNITED STATES' and (d_year = 1997 or d_year = 1998) and p_category = 'MFGR#14' group by d_year, s_city, p_brand order by d_year, s_city, p_brand;"
        ]


In [ ]:


# Criar um cliente BigQuery
client = bigquery.Client()

# Exemplo de consulta
query = """
SELECT *
FROM `data-avatar-409500.ssbsf10.supplier`
LIMIT 10
"""

# Executar a consulta
query_job = client.query(query)

# Obter resultados
results = query_job.result()

# Imprimir resultados
for row in results:
    print(row)


Row((209, 'Supplier#000000209', 'fpTboatC6ogrozMPApz2DXml', 'CHINA    0', 'CHINA', 'ASIA', '28-207-121-7721'), {'s_suppkey': 0, 's_name': 1, 's_address': 2, 's_city': 3, 's_nation': 4, 's_region': 5, 's_phone': 6})
Row((324, 'Supplier#000000324', 'QDsgOSozg jniYR2Hz', 'CHINA    0', 'CHINA', 'ASIA', '28-722-560-7023'), {'s_suppkey': 0, 's_name': 1, 's_address': 2, 's_city': 3, 's_nation': 4, 's_region': 5, 's_phone': 6})
Row((581, 'Supplier#000000581', 'X pNyEcNqxYwiP0gJ', 'CHINA    0', 'CHINA', 'ASIA', '28-415-978-2518'), {'s_suppkey': 0, 's_name': 1, 's_address': 2, 's_city': 3, 's_nation': 4, 's_region': 5, 's_phone': 6})
Row((1650, 'Supplier#000001650', '0R0dQzwyoH6vgewbsib', 'CHINA    0', 'CHINA', 'ASIA', '28-295-469-8581'), {'s_suppkey': 0, 's_name': 1, 's_address': 2, 's_city': 3, 's_nation': 4, 's_region': 5, 's_phone': 6})
Row((1674, 'Supplier#000001674', 'nepm7O58', 'CHINA    0', 'CHINA', 'ASIA', '28-396-137-7834'), {'s_suppkey': 0, 's_name': 1, 's_address': 2, 's_city': 3, 's

In [ ]:
# Sua consulta original
sua_consulta = """
SELECT *
FROM `data-avatar-409500.ssbsf10.supplier`
LIMIT 10
"""

# Configuração para evitar o cache
configuracao_query = bigquery.QueryJobConfig(use_query_cache=False)

# Execute a consulta com a configuração
query_job = client.query(sua_consulta, job_config=configuracao_query)

job_id = query_job.job_id

# Obtenha detalhes do job
job_details = client.get_job(job_id)
total_time = job_details.ended - job_details.created
print("Elapsed Time:", total_time)
print("Bytes Processados:", job_details.total_bytes_processed)

Elapsed Time: 0:00:00.248000
Bytes Processados: 9185101


In [ ]:
time_d_ms  = total_time / datetime.timedelta(milliseconds=1)

In [ ]:
time_d_ms

249.0

In [ ]:
job_details.total_bytes_processed

9185101

In [ ]:
for i in range(len(queries)):
  arr_elapsed_time = []
  arr_bytes_processados = []
  for _ in range(1,31):
    # Sua consulta original
    start_time = datetime.utcnow()
    sua_consulta = queries[i]

    # Configuração para evitar o cache
    configuracao_query = bigquery.QueryJobConfig(use_query_cache=False)

    # Execute a consulta com a configuração
    query_job = client.query(sua_consulta, job_config=configuracao_query)

    end_time = datetime.utcnow()

    job_id = query_job.job_id

    # Obtenha detalhes do job
    job_details = client.get_job(job_id)
    #total_time = job_details.ended - job_details.created
    #time_d_ms  = total_time / datetime.timedelta(milliseconds=1)
    elapsed_time = (end_time - start_time).total_seconds()


    bytes_processados = job_details.total_bytes_processed

    arr_elapsed_time.append(elapsed_time*1000)
    arr_bytes_processados.append(bytes_processados)
    #print("Elapsed Time:", total_time)
    #print("Bytes Processados:", job_details.total_bytes_processed)


    df = pd.DataFrame()

    lista_de_tuplas = list(zip(arr_elapsed_time, arr_bytes_processados))
    df = pd.DataFrame(
      lista_de_tuplas,
      columns=['elapsed_time_ms', 'bytes_processados']
    )

    df.to_csv('query_'+str(i)+'.csv')



In [ ]:
job_details.total_bytes_processed